In [2]:
#importation de packages
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import pandas as pd
import seaborn as sns
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# load data from drive
data = pd.read_csv('/content/drive/MyDrive/tomato_prediction.csv') 

comprehension des données et data preprocessing

In [4]:
# data preprocessing
print(data.head())
print(data.info())
print(data.isnull().sum())

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# check the replication
print(data.duplicated().sum())
print(data.shape)

# supprision de replication
data = data.drop_duplicates()
print(data.shape)

   humidity(%)   luminosity(lux)   PH   temperature_soil(degrees Celsius)  \
0           71              1600  6.5                                23.3   
1           75              2000  6.9                                25.8   
2           64              1200  6.1                                21.8   
3           69              1700  6.6                                24.5   
4           66              1300  6.3                                22.7   

    temperature_air(degrees Celsius)   water_flow(liter/hour)  \
0                               28.3                      2.2   
1                               30.8                      3.3   
2                               26.8                      1.9   
3                               29.5                      2.8   
4                               27.7                      2.4   

    water_needed(l)  
0               4.5  
1               5.2  
2               3.8  
3               4.1  
4               3.6  
<class 'pandas

In [5]:
#preparation of features and target for the data preprocessing
X=data.iloc[:,0:6]
X.head()
Y=data.iloc[:,-1]
print('X=\n',X.head(),'\n')
print('Y=\n',Y.head(),'\n')

X=
    humidity(%)   luminosity(lux)   PH   temperature_soil(degrees Celsius)  \
0           71              1600  6.5                                23.3   
1           75              2000  6.9                                25.8   
2           64              1200  6.1                                21.8   
3           69              1700  6.6                                24.5   
4           66              1300  6.3                                22.7   

    temperature_air(degrees Celsius)   water_flow(liter/hour)  
0                               28.3                      2.2  
1                               30.8                      3.3  
2                               26.8                      1.9  
3                               29.5                      2.8  
4                               27.7                      2.4   

Y=
 0    4.5
1    5.2
2    3.8
3    4.1
4    3.6
Name:  water_needed(l), dtype: float64 



In [6]:
# split the data into train and test sets
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)





#neural network model

In [7]:
def create_model(optimizer):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_dim=6),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer=optimizer, loss='mse')
    return model

# Perform hyperparameter tuning
optimizers = ['SGD', 'RMSprop', 'Adam', 'Adadelta']
mse_scores = []
r2_scores = []

for optimizer in optimizers:
    model = create_model(optimizer)
    model.fit(X_train_scaled, Y_train, epochs=100, verbose=0)
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(Y_test, y_pred)
    r2 = r2_score(Y_test, y_pred)
    mse_scores.append(mse)
    r2_scores.append(r2)




1/1 [==============================] - 0s 58ms/step


In [8]:

# Select the best optimizer based on MSE scores
best_optimizer = optimizers[np.argmin(mse_scores)]
print("Best optimizer: ", best_optimizer)


Best optimizer:  SGD


In [9]:
# Train the model with the best optimizer and a suitable learning rate
model = create_model(best_optimizer)
model.fit(X_train_scaled, Y_train, epochs=500, verbose=0, batch_size=32)

In [10]:
# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print("MSE: ", mse)
print("R2 Score: ", r2)


1/1 [==============================] - 0s 65ms/step
MSE:  0.019267239685396915
R2 Score:  0.9581624176793204


In [11]:
# Make predictions with the model
zone_1= np.array([76, 1600, 6.7, 27, 20.94, 2.2])
zone1_scaled = scaler.transform(zone_1.reshape(1, -1))
pred1 = model.predict(zone1_scaled)
zone_2 = np.array([70.3, 1900, 6.5, 34.7, 25.9, 2.2])
zone2_scaled = scaler.transform(zone_2.reshape(1, -1))
pred2 = model.predict(zone2_scaled)
print("Prediction1 for the zone number 1: ", pred1)
print("Prediction2 for the zone number 2: ", pred2)
print("In the zone, one 'large' is 0.5 hectare. To determine the water needed for this zone, we calculate 5500 * pred1:", 5500*pred1, "L")
print("In the zone two 'large' is 0.5 hectare. To determine the water needed for this zone, we calculate 5500 * pred2:", 5500*pred2, "L")

1/1 [==============================] - 0s 21ms/step
Prediction1 for the zone number 1:  [[2.0653613]]
Prediction2 for the zone number 2:  [[2.3451388]]
In the zone, one 'large' is 0.5 hectare. To determine the water needed for this zone, we calculate 5500 * pred1: [[11359.487]] L
In the zone two 'large' is 0.5 hectare. To determine the water needed for this zone, we calculate 5500 * pred2: [[12898.264]] L


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [12]:
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                448       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2,561
Trainable params: 2,561
Non-trainable params: 0
_________________________________________________________________


*Donlawd_the_model*

In [13]:
# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model as a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

# Download the TensorFlow Lite model file
from google.colab import files
#files.download('model.tflite')

In [14]:
import tensorflow as tf
from google.colab import files

# Save the model as TensorFlow SavedModel
tf.saved_model.save(model, 'saved_model')

# Download the SavedModel .pb file
#files.download('saved_model/saved_model.pb')
